### Import Libraries

In [28]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [29]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET","/summary",payload,headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

### Convert Data to Json

In [30]:
covid1=json.loads(data)

### Normalise Data

In [31]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,14,39268,0,1458,43,32789,2020-10-01T12:57:04Z
1,Albania,AL,albania,131,13649,3,387,115,7847,2020-10-01T12:57:04Z
2,Algeria,DZ,algeria,162,51530,10,1736,111,36174,2020-10-01T12:57:04Z
3,Andorra,AD,andorra,84,2050,0,53,167,1432,2020-10-01T12:57:04Z
4,Angola,AO,angola,67,4972,4,183,108,1941,2020-10-01T12:57:04Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,0,1094,0,35,3,1010,2020-10-01T12:57:04Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-01T12:57:04Z
185,Yemen,YE,yemen,3,2034,0,587,11,1286,2020-10-01T12:57:04Z
186,Zambia,ZM,zambia,44,14759,0,332,22,13959,2020-10-01T12:57:04Z


### Convert to Pandas DataFrame

In [32]:
df=pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,14,39268,0,1458,43,32789,2020-10-01T12:57:04Z,{}
1,Albania,AL,albania,131,13649,3,387,115,7847,2020-10-01T12:57:04Z,{}
2,Algeria,DZ,algeria,162,51530,10,1736,111,36174,2020-10-01T12:57:04Z,{}
3,Andorra,AD,andorra,84,2050,0,53,167,1432,2020-10-01T12:57:04Z,{}
4,Angola,AO,angola,67,4972,4,183,108,1941,2020-10-01T12:57:04Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,0,1094,0,35,3,1010,2020-10-01T12:57:04Z,{}
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-01T12:57:04Z,{}
185,Yemen,YE,yemen,3,2034,0,587,11,1286,2020-10-01T12:57:04Z,{}
186,Zambia,ZM,zambia,44,14759,0,332,22,13959,2020-10-01T12:57:04Z,{}


### Drop Unnecessary Columns

In [33]:
covid2 = df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,14,39268,0,1458,43,32789
1,Albania,131,13649,3,387,115,7847
2,Algeria,162,51530,10,1736,111,36174
3,Andorra,84,2050,0,53,167,1432
4,Angola,67,4972,4,183,108,1941
...,...,...,...,...,...,...,...
183,Viet Nam,0,1094,0,35,3,1010
184,Western Sahara,0,10,0,1,0,8
185,Yemen,3,2034,0,587,11,1286
186,Zambia,44,14759,0,332,22,13959


### Generate Base Map

In [34]:
m = folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
m

### Obtain Geodata

In [35]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'


### Generate Choropleth Map Layer

In [36]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Covid19',
    data=covid2,
    columns = ['Country','TotalConfirmed',],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = 'Total Confirmed Covid Cases',
).add_to(m)
m

### Generate Circular Markers

In [37]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [38]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [39]:
covid_final = pd.merge(covid2,coordinates,on='Country')

In [40]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                       fill_color='#000000').add_to(m)

In [41]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m

### Generate Base Map

In [42]:
m1 = folium.Map(tiles='StamenToner',min_zoom=2)
m1

### Generate Heat Map Layer

In [43]:
deaths = covid_final['TotalDeaths'].astype(float)

In [44]:
lat = covid_final['latitude'].astype(float)

In [45]:
lon = covid_final['longitude'].astype(float)

In [46]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

### Generate Circular Marker

In [47]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalRecovered],
                       fill_color="#000000").add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1